In [1]:
%pip install -q ./arc/arcadia/dataforge/

Note: you may need to restart the kernel to use updated packages.


In [3]:
from crowdom import bots

import yaml
import logging.config
from typing import Dict

In [ ]:
with open('logging.yaml') as f:
    logging.config.dictConfig(yaml.full_load(f.read()))

In [ ]:
register = bots.NDARegisterClient()

In [4]:
admin = register.create_admin()
admin_client = bots.NDATolokaClient(admin['oauth'])

In [5]:
requester = register.create_user(user_type='requester')
requester_client = bots.NDATolokaClient(requester['oauth'])

In [6]:
admin_client.change_credit(requester['id'], 100)

{'success': True}

In [7]:
from datetime import timedelta

from crowdom import base, client, datasource, objects

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
class Animal(base.Class):
    DOG = 'dog'
    CAT = 'cat'
    OTHER = 'other'

    @classmethod
    def labels(cls) -> Dict['Animal', Dict[str, str]]:
        return {
            cls.DOG: {'EN': 'dog', 'RU': 'собака'},
            cls.CAT: {'EN': 'cat', 'RU': 'кошка'},
            cls.OTHER: {'EN': 'other', 'RU': 'другое'}}

In [10]:
function = base.ClassificationFunction(inputs=(objects.Image,), cls=Animal)

In [11]:
task_duration_hint = timedelta(seconds=10)

In [12]:
task_spec_p = client.PreparedTaskSpec(
    task_spec=base.TaskSpec(
        id='id',
        function=function,
        name=base.EMPTY_STRING,
        description=None,
        instruction=None),
    lang=None)

In [15]:
solved_tasks = datasource.read_tasks('bots_tasks.json', task_spec_p.task_mapping, has_solutions=True)

In [16]:
worker_types = [
    (bots.WorkerConfig(bots.WorkerSpeed.SLOW, 0.9, Animal), 4),
    (bots.WorkerConfig(bots.WorkerSpeed.SLOW, 0.75, Animal), 2),
    (bots.WorkerConfig(bots.WorkerSpeed.MEDIUM, 0.85, Animal), 6),
    (bots.WorkerConfig(bots.WorkerSpeed.MEDIUM, 0.7, Animal), 8),
    (bots.WorkerConfig(bots.WorkerSpeed.FAST, 0.85, Animal), 4),
    (bots.WorkerConfig(bots.WorkerSpeed.FAST, 0.6, Animal), 6),
    (bots.WorkerConfig(bots.WorkerSpeed.POOR, 0.4, Animal), 10),
    (bots.WorkerConfig(bots.WorkerSpeed.RAND, 0.2, Animal), 5)]

In [18]:
solutions = bots.create_tasks_info_dict(task_spec_p.task_mapping, solved_tasks)

In [19]:
pool_id = ...

In [21]:
wg = bots.WorkerGroup(
    pool_id,
    task_spec_p.task_mapping,
    task_duration_hint,
    worker_types,
    solutions)

In [22]:
wg.start()

In [23]:
# after loop finishes:

In [24]:
wg.stop()
